In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from scipy import stats
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import math
import calendar
from matplotlib import pyplot as plt
%pylab inline
warnings.filterwarnings("ignore")
os.chdir(r'''D:\Coursera_ML\Final Project\Data''')

C:\Program Files\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Populating the interactive namespace from numpy and matplotlib


#### 1. Загрузка данных.

In [2]:
data_january=pd.read_excel(r'''Grouped_data_january.xlsx''')
data_february=pd.read_excel(r'''Grouped_data_february.xlsx''')
data_march=pd.read_excel(r'''Grouped_data_march.xlsx''')
data_april=pd.read_excel(r'''Grouped_data_april.xlsx''')
data_may=pd.read_excel(r'''Grouped_data_may.xlsx''')
data_june=pd.read_excel(r'''Grouped_data_june.xlsx''')

In [3]:
regions=(data_may.groupby('region').sum()['N_trips']/24/31)[data_may.groupby('region').sum()['N_trips']/24/31>=5].index.values

In [4]:
data_january=data_january[data_january.region.apply(lambda x: x in regions)]
data_february=data_february[data_february.region.apply(lambda x: x in regions)]
data_march=data_march[data_march.region.apply(lambda x: x in regions)]
data_april=data_april[data_april.region.apply(lambda x: x in regions)]
data_may=data_may[data_may.region.apply(lambda x: x in regions)]
data_june=data_june[data_june.region.apply(lambda x: x in regions)]

In [5]:
data_train=pd.concat([data_january,data_february,data_march,data_april])
data_train.index=data_train.pickup_datehour

In [6]:
data_train_may=pd.concat([data_january,data_february,data_march,data_april,data_may])
data_train_may.index=data_train_may.pickup_datehour

#### 2. Преобразование данных.

In [7]:
aggregated_data=pd.DataFrame(np.array([str(x)+'_'+str(y) for x in data_train.pickup_datehour.unique() for y in regions]).reshape(len(data_train.pickup_datehour.unique()),len(regions)),columns=regions,index=data_train.pickup_datehour.unique()).applymap(lambda x: x.split('_'))

In [8]:
aggregated_data_may=pd.DataFrame(np.array([str(x)+'_'+str(y) for x in data_train_may.pickup_datehour.unique() for y in regions]).reshape(len(data_train_may.pickup_datehour.unique()),len(regions)),columns=regions,index=data_train_may.pickup_datehour.unique()).applymap(lambda x: x.split('_'))

In [9]:
def vlookup(data):
    def vlp(x):
        try:
            val=data.loc[x[0],:][data.loc[x[0],:].region==int(x[1])]['N_trips'][0]
        except:
            val=0
        return val
    return vlp

In [10]:
aggregated=aggregated_data.applymap(vlookup(data_train))

In [11]:
aggregated_may=aggregated_data_may.applymap(vlookup(data_train_may))

In [12]:
aggregated_stand=aggregated.apply(lambda x: (x-np.mean(x))/np.std(x,ddof=1))

In [13]:
aggregated_stand_may=aggregated_may.apply(lambda x: (x-np.mean(x))/np.std(x,ddof=1))

In [14]:
T=np.arange(1,len(aggregated_stand)+1)
exog=pd.DataFrame()
K=5
for i in range(1,K+1):
    exog['sin_'+str(i)]=np.sin(T*2*math.pi*i/168)
    exog['cos_'+str(i)]=np.cos(T*2*math.pi*i/168)
exog['Intercept']=1

In [15]:
T=np.arange(1,len(aggregated_stand_may)+1)
exog_may=pd.DataFrame()
K=5
for i in range(1,K+1):
    exog_may['sin_'+str(i)]=np.sin(T*2*math.pi*i/168)
    exog_may['cos_'+str(i)]=np.cos(T*2*math.pi*i/168)
exog_may['Intercept']=1

In [16]:
data_june.index=data_june.pickup_datehour
aggregated_data_june=pd.DataFrame(np.array([str(x)+'_'+str(y) for x in data_june.pickup_datehour.unique() for y in regions]).reshape(len(data_june.pickup_datehour.unique()),len(regions)),columns=regions,index=data_june.pickup_datehour.unique()).applymap(lambda x: x.split('_'))
aggregated_june=aggregated_data_june.applymap(vlookup(data_june))
aggregated_stand_june=aggregated_june.apply(lambda x: (x-np.mean(x))/np.std(x,ddof=1))
T_june=np.arange(len(exog_may)+1,len(exog_may)+1+len(aggregated_stand_june))
exog_june=pd.DataFrame()
K=5
for i in range(1,K+1):
    exog_june['sin_'+str(i)]=np.sin(T_june*2*math.pi*i/168)
    exog_june['cos_'+str(i)]=np.cos(T_june*2*math.pi*i/168)
exog_june['Intercept']=1
exog_final=pd.concat([exog_may,exog_june])
exog_final.index=np.arange(len(exog_final))
aggregated_stand_final=pd.concat([aggregated_stand_may,aggregated_stand_june])
aggregated_final=pd.concat([aggregated_may,aggregated_june])

#### 3. Загрузка SARIMA прогнозов.

In [17]:
preds_csv=pd.read_csv(r'''TRUE_PREDS_ALL.csv''',engine='python')

In [18]:
preds_csv.head()

,id,y
0,1075_2016-01-03_23_1,42.080105
1,1075_2016-01-04_0_1,7.405356
2,1075_2016-01-04_1_1,0.000000
3,1075_2016-01-04_2_1,0.000000
4,1075_2016-01-04_3_1,0.000000


#### 4. Создание признаков.

In [19]:
exog_final.index=aggregated_final.index

##### Возьмем день недели и час из второго подпункта первого пункта задания как категориальные признаки - остальные признаки этого подпункта логически не имеют смысла.

In [85]:
import calendar
def w_day(x):
    date=x.split(' ')[0]
    y,m,d=np.array(date.split('-')).astype(int)
    res=calendar.weekday(y,m,d)
    return res
exog_final['week_day']=pd.Series(exog_final.index,index=exog_final.index).apply(w_day)
exog_final['hour']=pd.Series(exog_final.index,index=exog_final.index).apply(lambda x: int(x.split(' ')[1].split(':')[0]))

##### Возьмем $K=6$ и $K_d=2$, так как это рекомендовалось в задании.

In [21]:
shifted=pd.concat([pd.DataFrame(aggregated_final.shift(periods=i).values,\
                                columns=aggregated_final.columns.astype(str)+\
                                np.array(['_'+str(i)]*len(aggregated_final.columns)),\
            index=aggregated_final.index) for i in range(1,7)],axis=1)
shifted_seasonal=pd.concat([pd.DataFrame(aggregated_final.shift(periods=i*24).values,\
                                columns=aggregated_final.columns.astype(str)+\
                                np.array(['_'+str(i*24)]*len(aggregated_final.columns)),\
            index=aggregated_final.index) for i in range(1,3)],axis=1)
shifted_all=pd.concat([shifted,shifted_seasonal],axis=1)

##### Возьмем суммарное количество поездок за предшествующиe полдня и день. За месяц и неделю не будем брать, это выкинет много начальных наблюдений.

In [22]:
def outer_periods(p):
    def sum_periods(x):
        pos=(aggregated_final.index==x.name).argmax()
        if pos<p: return pd.Series([np.nan]*len(x),index=aggregated_final.columns)
        else:
            ind=np.arange(pos-p,pos,1)
            return aggregated_final.iloc[ind,:].apply(sum)
    return sum_periods

In [23]:
sum_day=aggregated_final.apply(outer_periods(24),axis=1)

In [24]:
sum_halfday=aggregated_final.apply(outer_periods(12),axis=1)

In [25]:
import datetime
from datetime import datetime as dt

In [1]:
def recover_ind(x):
    date,hour,step=x.split('_')[1:]
    if len(hour)==1: hour='0'+hour
    time=hour+':00:00'
    actual=str(dt.strptime(date+' '+time,'%Y-%m-%d %H:%M:%S')+datetime.timedelta(hours=int(step)))
    return actual

In [ ]:
preds_csv.index=preds_csv.id.apply(recover_ind)

In [28]:
preds_csv['step']=preds_csv.id.apply(lambda x: int(x.split('_')[-1]))

In [29]:
preds_csv['reg']=preds_csv.id.apply(lambda x: int(x.split('_')[0]))

In [247]:
preds_csv=preds_csv.drop(['pred_data'],axis=1)

In [249]:
import random
total_inds=pd.Series(np.nan,index=aggregated_final.index)

def extract_preds(reg,step):
    data=preds_csv[(preds_csv['reg']==reg)&(preds_csv['step']==step)][['y','id']]
    res=total_inds
    res[data.index]=data['y']
    return res


In [98]:
for val in exog_final.week_day.unique()[1:]:
    exog_final['week_day'+str(val)]=exog_final.week_day.apply(lambda x: int(x==val))
for val in exog_final.hour.unique()[1:]:
    exog_final['hour'+str(val)]=exog_final.hour.apply(lambda x: int(x==val))

In [101]:
exog_final=exog_final.drop(['week_day','hour'],axis=1)

In [102]:
def mdot(*args):
    st=args[0]
    for val in args[1:]:
        st=np.dot(st,val)
    return st

In [ ]:
from sklearn.linear_model import Lasso, Ridge
def outer_linreg(estimator,lambd,ind,ind2,mode='score'):
    def linreg(x):
        reg=x.name
        colnames=shifted_all.columns[pd.Series(shifted_all.columns).apply(lambda x: x.split('_')[0]==str(reg))]
        ar=shifted_all[colnames]
        res=np.array([])
        if mode=='score':
            for step in range(1,7):
                preds=extract_preds(reg,step)
                X_all=pd.concat([exog_final,sum_halfday[[reg]],sum_day[[reg]],ar,preds],axis=1)
                X=X_all.iloc[:ind,:].dropna(axis=0)
                y=x[X.index].values
                X=X.values
                model=eval(estimator)(alpha=lambd).fit(X,y)
                X_test=X_all.iloc[ind:ind2,:].dropna(axis=0)
                y_test=x[X_test.index].values
                X_test=X_test.values
                prd=mdot(X_test,model.coef_)
                res=np.append(res,np.abs(y_test-prd))
            val=np.sum(res)
            return val
        elif mode=='data':
            new_preds=pd.DataFrame()
            for step in range(1,7):
                preds=extract_preds(reg,step)
                X_all=pd.concat([exog_final,sum_halfday[[reg]],sum_day[[reg]],ar,preds],axis=1)
                X=X_all.iloc[:ind,:].dropna(axis=0)
                y=x[X.index].values
                X=X.values
                model=eval(estimator)(alpha=lambd).fit(X,y)
                X_test=X_all.iloc[ind+step-1:ind2-6+step,:].dropna(axis=0)
                data=preds_csv[(preds_csv['reg']==reg)&(preds_csv['step']==step)].loc[list(X_test.index),['id']]
                y_test=x[X_test.index].values
                X_test=X_test.values
                prd=mdot(X_test,model.coef_)
                data['y']=prd
                res=np.append(res,np.abs(y_test-prd))
                new_preds=pd.concat([new_preds,data])
            val=np.sum(res)
            return val,new_preds         
            
    return linreg

##### Проведем оценку параметров регрессии c L1 и L2 регуляризаторами для каждого региона в отдельности. На данных до апреля включительно необходимо ответить на два вопроса: 1) какой регуляризатор ''лучше''? 2) каким взять $\lambda$?. Иными словами, необходимо настроить гиперпараметры. Рассмотрим сетку  $\lambda \in \{1,5,10,20\}$.

In [290]:
from itertools import product

In [299]:
params=product(['Ridge','Lasso'],[1,5,10,20])

In [300]:
vals={}
for method,lambd in params:
    f=outer_linreg(method,lambd,len(exog),len(exog_may),mode='score')
    res=sum(aggregated_final.apply(f))/102/739/6
    vals[(method,lambd)]=res

##### Ниже приведены средние абсолютные отклонения в мае для моделей, обученных на данных до апреля. Неделю назад $MAE\approx 26$, значит каждая, из нижеприведенных моделей показывает лучший результат.

In [301]:
vals

{('Lasso', 1): 21.097122121541116,
 ('Lasso', 5): 22.815161260498083,
 ('Lasso', 10): 23.106258417162994,
 ('Lasso', 20): 23.350874330272944,
 ('Ridge', 1): 23.604211311053366,
 ('Ridge', 5): 21.68516382310419,
 ('Ridge', 10): 21.28045332348924,
 ('Ridge', 20): 21.050894457842634}

##### Отберем две модели: ('Lasso', 1) и ('Ridge', 20), оценим их на данных до мая и построим прогнозы на июнь.

In [305]:
from functools import reduce
vals_june={}
vals_june_data={}
for method,lambd in [('Lasso', 1),('Ridge', 20)]:
    f=outer_linreg(method,lambd,len(exog_may),len(exog_final),mode='data')
    estimates=aggregated_final.apply(f)
    errors=estimates.apply(lambda x: x[0])
    hats=estimates.apply(lambda x: x[1])
    preds=reduce(lambda x,y:pd.concat([x,y]),hats)
    res=sum(errors)/102/715/6
    vals_june[(method,lambd)]=res
    vals_june_data[(method,lambd)]=preds

##### Ниже результаты, и они практически одинаковы. Выберем Лассо.

In [306]:
vals_june

{('Lasso', 1): 20.13099501584597, ('Ridge', 20): 20.140670551020992}

In [312]:
vals_june_data[('Lasso', 1)].to_csv('Lasso_preds.csv',index=False)

# Ссылка на сабмишн.
https://www.kaggle.com/c/yellowtaxi/submissions?sortBy=date&group=all&page=1